In [1]:
#TextToExcel

In [2]:
import os
import pandas as pd
import numpy as np
from openpyxl.styles import Alignment
from openpyxl.utils import get_column_letter

# Function to get all directories starting with 'GA'
def get_ga_directories(parent_folder):
    return [d for d in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, d)) and d.startswith('GA')]

# Set the parent folder where all GA directories are located
parent_folder = '.'  # Change this to the actual path if needed

# Get the list of top-level GA directories
ga_directories = get_ga_directories(parent_folder)

def try_convert_numeric(val):
    try:
        return pd.to_numeric(val)
    except ValueError:
        return val

# Process each GA directory
for ga_dir in ga_directories:
    ga_path = os.path.join(parent_folder, ga_dir)
    excel_file = os.path.join(parent_folder, f'{ga_dir}.xlsx')
    
    # Get all subdirectories in the GA directory
    subdirectories = [d for d in os.listdir(ga_path) if os.path.isdir(os.path.join(ga_path, d)) and d.startswith(ga_dir)]
    
    # Initialize an Excel writer for this GA
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        # Process each subdirectory (experiment)
        for subdir in subdirectories:
            combined_df = pd.DataFrame()
            
            # Loop over each file in the subdirectory
            for i in range(1, 11):
                file_name = os.path.join(ga_path, subdir, f'{subdir}-Result{i}.txt')
                column_name = f'{i}'
                
                if os.path.isfile(file_name):
                    try:
                        with open(file_name, 'r') as file:
                            content = file.read().splitlines()
                        
                        # Create a DataFrame and attempt to convert to numeric
                        df = pd.DataFrame({column_name: content})
                        df[column_name] = df[column_name].apply(try_convert_numeric)
                        
                        combined_df = pd.concat([combined_df, df], axis=1)
                    except Exception as e:
                        print(f"Error reading file {file_name}: {e}")
                        continue
                else:
                    print(f"File not found: {file_name}")
                    continue
            
            if not combined_df.empty:
                try:
                    # Ensure numeric columns are written as numbers
                    combined_df = combined_df.apply(pd.to_numeric)
                    
                    # Create the 'Generation \ Experiment' column
                    gen_exp_column = list(range(1, 2001))  # First 2000 rows
                    gen_exp_column.append('')  # Skip 1 line
                    gen_exp_column.append('Values')  # Add 'Values'
                    gen_exp_column.extend(range(1, 31))  # Add 1 to 30
                    gen_exp_column.append('')  # Skip 1 line
                    gen_exp_column.append('Time')  # Add 'Time'
                    gen_exp_column.append('')  # Skip 1 line
                    
                    # Create a new DataFrame with the correct number of rows
                    new_df = pd.DataFrame(index=range(len(gen_exp_column)))
                    
                    # Add the 'Generation \ Experiment' column
                    new_df['Generation \\ Experiment'] = gen_exp_column
                    
                    # Add data from combined_df, filling with empty strings where necessary
                    for col in combined_df.columns:
                        new_df[col] = combined_df[col].tolist() + [''] * (len(new_df) - len(combined_df))
                    
                    # Write to Excel
                    new_df.to_excel(writer, sheet_name=subdir, index=False)
                    
                    # Get the worksheet
                    worksheet = writer.sheets[subdir]
                    
                    # Add headers for min, max, and average
                    worksheet.cell(row=2000, column=12, value='Min')
                    worksheet.cell(row=2000, column=13, value='Max')
                    worksheet.cell(row=2000, column=14, value='Average')
                    
                    worksheet.cell(row=2034, column=12, value='Min')
                    worksheet.cell(row=2034, column=13, value='Max')
                    worksheet.cell(row=2034, column=14, value='Average')
                    
                    # Calculate and write min, max, and average for rows 2001-2030
                    
                    data_range = [cell.value for cell in worksheet[2001][1:11]]  # Columns B to K
                    min_val = min(data_range)
                    max_val = max(data_range)
                    avg_val = sum(data_range) / len(data_range)

                    # Format the values to 6 decimal places
                    min_val = f"{min_val:.6f}"
                    max_val = f"{max_val:.6f}"
                    avg_val = f"{avg_val:.6f}"
                        
                    worksheet.cell(row=2001, column=12, value=min_val)
                    worksheet.cell(row=2001, column=13, value=max_val)
                    worksheet.cell(row=2001, column=14, value=avg_val)

                    data_range = [cell.value for cell in worksheet[2035][1:11]]  # Columns B to K
                    min_val = min(data_range)
                    max_val = max(data_range)
                    avg_val = sum(data_range) / len(data_range)

                    # Format the values to 6 decimal places
                    min_val = f"{min_val:.6f}"
                    max_val = f"{max_val:.6f}"
                    avg_val = f"{avg_val:.6f}"
                        
                    worksheet.cell(row=2035, column=12, value=min_val)
                    worksheet.cell(row=2035, column=13, value=max_val)
                    worksheet.cell(row=2035, column=14, value=avg_val)
                    
                    
                    # Adjust column widths and alignment
                    for col in range(1, 15):  # Columns A to N
                        max_len = max(len(str(cell.value)) for cell in worksheet[get_column_letter(col)])
                        worksheet.column_dimensions[get_column_letter(col)].width = max_len + 2
                        
                        # Center align all columns except the first one
                        if col > 1:
                            for cell in worksheet[get_column_letter(col)]:
                                cell.alignment = Alignment(horizontal='center')
                    
                except Exception as e:
                    print(f"Error writing to Excel sheet {subdir}: {e}")
            else:
                print(f"No files found for directory: {subdir}")
    
    print(f"Created Excel file: {excel_file}")

print("Done processing all GA directories.")

Created Excel file: .\GA1.xlsx
Created Excel file: .\GA2.xlsx
Created Excel file: .\GA3.xlsx
Done processing all GA directories.


In [3]:
#decimal places, highlight average

In [4]:
import os
import openpyxl
from openpyxl.styles import PatternFill

def format_cells_in_excel(directory, cell_ranges):
    # Iterate through all files in the specified directory
    for filename in os.listdir(directory):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(directory, filename)
            
            # Load the workbook and iterate through each sheet
            workbook = openpyxl.load_workbook(file_path)
            for sheet in workbook.worksheets:
                
                # Format each specified range of cells
                for start_cell, end_cell in cell_ranges:
                    for row in sheet[start_cell:end_cell]:
                        for cell in row:
                            if isinstance(cell.value, (int, float)):
                                cell.number_format = '0.000000'
                
                # Adjust the width of all columns in the sheet
                for column_cells in sheet.columns:
                    max_length = 0
                    column_letter = column_cells[0].column_letter  # Get the column letter
                    for cell in column_cells:
                        if cell.value is not None:
                            # Calculate the length of the cell content
                            cell_length = len(str(cell.value))
                            if cell_length > max_length:
                                max_length = cell_length
                    # Adjust column width to fit the longest content plus some buffer
                    adjusted_width = max_length + 10  # Adding buffer for readability
                    sheet.column_dimensions[column_letter].width = adjusted_width
            
            # Save the workbook after processing all sheets
            workbook.save(file_path)
            print(f"Formatted cells and adjusted column widths in {filename}")

# Example usage:
directory_path = "."  # Refers to the current directory

# List of cell ranges to format
cell_ranges = [
    ("B2", "K2001"),
    ("L2001", "N2001"),
    ("B2004", "K2033"),
    ("B2035", "N2035")
]

format_cells_in_excel(directory_path, cell_ranges)

def highlight_cells_in_excel(directory, cell_addresses, color="FFFF00"):
    # Define the fill color
    fill = PatternFill(start_color=color, end_color=color, fill_type="solid")
    
    # Iterate through all files in the specified directory
    for filename in os.listdir(directory):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(directory, filename)
            
            # Load the workbook and iterate through each sheet
            workbook = openpyxl.load_workbook(file_path)
            for sheet in workbook.worksheets:
                
                # Highlight the specified cells in the current sheet
                for cell_address in cell_addresses:
                    cell = sheet[cell_address]
                    cell.fill = fill
                
            # Save the workbook after processing all sheets
            workbook.save(file_path)
            print(f"Highlighted cells {cell_addresses} in {filename}")

# Example usage:
directory_path = "."  # Refers to the current directory

# List of cell addresses to be highlighted
cell_addresses = ["N2000", "N2034"]  # Specify the cells to highlight

highlight_cells_in_excel(directory_path, cell_addresses)

Formatted cells and adjusted column widths in GA1.xlsx
Formatted cells and adjusted column widths in GA2.xlsx
Formatted cells and adjusted column widths in GA3.xlsx
Highlighted cells ['N2000', 'N2034'] in GA1.xlsx
Highlighted cells ['N2000', 'N2034'] in GA2.xlsx
Highlighted cells ['N2000', 'N2034'] in GA3.xlsx


In [5]:
#summarise

In [6]:
import os
import openpyxl

def copy_values_to_text_file(directory, output_file, cells):
    with open(output_file, 'w') as file:
        # Iterate through all files in the specified directory
        for filename in os.listdir(directory):
            if filename.endswith(".xlsx"):
                file_path = os.path.join(directory, filename)
                workbook = openpyxl.load_workbook(file_path)
                
                # Write the name of the Excel file to the text file
                file.write(f"[{filename}]\n")
                
                # Iterate through each sheet in the workbook
                for sheet_index, sheet in enumerate(workbook.worksheets):
                    # Write the name of the sheet
                    file.write(f"{sheet.title}\n")
                    
                    # Extract and write the values from the specified cells
                    for label, cell_address in cells:
                        cell_value = sheet[cell_address].value
                        file.write(f"{label}\n")
                        file.write(f"{cell_value}\n")
                    
                    # Add a newline to separate each sheet's content
                    if sheet_index < len(workbook.worksheets) - 1:
                        file.write("\n")
                
                # Add a dashed line to separate each workbook's content
                file.write("-" * 50 + "\n\n")
                
    print(f"Copied values to {output_file}")

# Example usage:
directory_path = "."  # Refers to the current directory
output_text_file = "output_values.txt"  # Output text file

# List of cells and their labels
cells_to_copy = [
    ("Average Fitness", "N2001"),
    ("Average Time", "N2035")
]

copy_values_to_text_file(directory_path, output_text_file, cells_to_copy)

Copied values to output_values.txt
